In [5]:
import os
import threading
import time
import chromadb
import torch
import requests
import nest_asyncio

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from rank_bm25 import BM25Okapi
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer

# Initialize FastAPI
app = FastAPI()

# Load tokenizer for truncation
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Truncate context based on token limit
def truncate_text_to_token_limit(text, max_tokens=512):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return tokenizer.convert_tokens_to_string(tokens)

# Load LLM (FLAN-T5)
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={
        "do_sample": True,
        "temperature": 0.7,
        "max_new_tokens": 256
    }
)


# Path to vector store
vector_store_path = "vector_store"

# Load documents from PDF
def load_documents(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    documents = text_splitter.split_documents(pages)
    return [doc.page_content for doc in documents]

# Create and save vector store
def create_and_save_vector_store(texts, vector_store_path):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = Chroma.from_texts(texts, embedding=embeddings, persist_directory=vector_store_path)
    vector_store.persist()
    return vector_store

# Load vector store
def load_vector_store(vector_store_path):
    if os.path.exists(vector_store_path):
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        return Chroma(persist_directory=vector_store_path, embedding_function=embeddings)
    return None

# Create BM25 index
def create_bm25_index(texts):
    tokenized_corpus = [text.split() for text in texts]
    return BM25Okapi(tokenized_corpus), texts

# Load and process data
pdf_path = r"C:\Users\mouni\tun_law_project\Tunisia_Crim-Fr.pdf"
texts = load_documents(pdf_path)

vector_store = load_vector_store(vector_store_path)
if vector_store is None:
    vector_store = create_and_save_vector_store(texts, vector_store_path)

bm25, bm25_texts = create_bm25_index(texts)

# Request schema
class QueryRequest(BaseModel):
    query: str

# Endpoint
@app.post("/query")
def query_chatbot(request: QueryRequest):
    query = request.query

    # BM25 retrieval
    bm25_scores = bm25.get_scores(query.split())
    top_bm25 = [bm25_texts[i] for i in torch.topk(torch.tensor(bm25_scores), 3).indices.tolist()]

    # Chroma dense retrieval
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    chroma_results = retriever.get_relevant_documents(query)
    top_dense = [doc.page_content for doc in chroma_results]

    # Combine and truncate context
    combined_context = "\n".join(top_bm25 + top_dense)
    truncated_context = truncate_text_to_token_limit(combined_context, max_tokens=512)

    # Prompt (refined for French legal question answering)
    prompt = (
        f"Voici un extrait de texte juridique :\n{truncated_context}\n\n"
        f"Question : {query}\n"
        f"Répondez de manière complète et précise en vous basant uniquement sur le texte ci-dessus."
    )

    # Generate answer
    response = llm(prompt)
    answer = response[0]['generated_text'] if isinstance(response[0], dict) else response[0]

    return {
        "answer": answer.strip(),
        "citations": top_bm25 + top_dense
    }

# Apply nest_asyncio
nest_asyncio.apply()

# Run app and test queries
if __name__ == "__main__":
    import uvicorn

    def run_server():
        uvicorn.run(app, host="127.0.0.1", port=8001)

    # Run server in a thread
    server_thread = threading.Thread(target=run_server)
    server_thread.start()

    # Wait for the server to start
    time.sleep(3)

    # Sample queries
    queries = [
        "Que signifie l'article 19 concernant les dommages-intérêts après une condamnation ?",
        "Que se passe-t-il si le condamné n'a pas assez de biens pour payer l'amende ?",
        "Est-ce que plusieurs condamnés dans la même affaire doivent payer ensemble ?"
    ]

    for q in queries:
        response = requests.post("http://127.0.0.1:8001/query", json={"query": q})
        print(f"\nQuestion : {q}")
        print("Réponse :", response.json()["answer"])
        print("Citations :", response.json()["citations"])


Device set to use cpu
INFO:     Started server process [11108]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8001): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
Task exception was never retrieved
future: <Task finished name='Task-28' coro=<Server.serve() done, defined at c:\Users\mouni\anaconda3\envs\tun_law_env\lib\site-packages\uvicorn\server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "c:\Users\mouni\anaconda3\envs\tun_law_env\lib\site-packages\uvicorn\server.py", line 163, in startup
    server = await loop.create_server(
  File "c:\Users\mouni\anaconda3\envs\tun_law_env\lib\asyncio\base_events.py", line 1506, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 10048] erro


Question : Que signifie l'article 19 concernant les dommages-intérêts après une condamnation ?
Réponse : L'acquittement, or la condamnation aux peines édictées par la loi, is penalized without prejudice to the restitutions and  damages to the parties lesées
Citations : ["de relaxe, soit après ordonnance ou arrêt de non-lieu émanant \ndu juge d’instruction, soit après classement de la dénonciation \npar le magistrat, fonctionnaire, autorité concernée ou employeur \nhabilité à apprécier la suite à donner à la dénonciation. \nLa juridiction saisie en vertu du présent article est tenue de surseoir à \nstatuer si des poursuites concernant le fait dénoncé sont pendantes. \nArticle 249 \nNe peut être retenu comme excuse, le fait d'arguer que les", "Imprimerie Officielle de la République Tunisienne\n19 \nLa peine est de dix ans d'emprisonnement s’il n’est pas \nétabli que les receleurs étaient en connaissance des \ncirconstances qui ont justifié la condamnation des auteurs \nprincipaux à la p

In [ ]:
import os
import torch
import logging
import sys
import traceback

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Model configuration
LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Changed to a publicly available model

# Path to vector store
vector_store_path = "vector_store"

# Embedding model configuration (changed to a model with 768 dimensions)
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"

# Model loading functions
def load_llm():
    """Load the language model."""
    logger.info(f"Loading language model: {LLM_MODEL}")
    
    try:
        # Check for available device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Using device: {device}")
        
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
        
        # Check if the tokenizer has pad_token set
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            logger.info("Set pad_token to eos_token as it was None")
            
        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None,
            trust_remote_code=True  # Sometimes needed for custom models
        )
        
        # Create text generation pipeline with safer parameters
        gen_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,  # Reduced from 512
            do_sample=True,
            temperature=0.5,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            device=0 if device == "cuda" else -1
        )

        logger.info("LLM loaded successfully")
        return HuggingFacePipeline(pipeline=gen_pipeline)
    except Exception as e:
        logger.error(f"Error loading LLM: {e}")
        logger.error(traceback.format_exc())
        raise

def create_legal_prompt_template():
    """Create a refined prompt template for legal QA."""
    logger.info("Creating legal QA prompt template.")
    # Simplified template that might work better with TinyLlama
    template = """
    <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: {context}
    Question: {question}
    </|user|>
    
    <|assistant|>
    """
    return PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

# Truncate context based on token limit
def truncate_text_to_token_limit(text, max_tokens=256):  # Reduced from 512
    try:
        tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
        encoded = tokenizer.encode(text)
        if len(encoded) > max_tokens:
            encoded = encoded[:max_tokens]
        return tokenizer.decode(encoded)
    except Exception as e:
        logger.error(f"Error truncating text: {e}")
        # Fallback to simple character truncation
        return text[:max_tokens * 4]

# Load documents from PDF
def load_documents(pdf_path):
    logger.info(f"Loading documents from {pdf_path}")
    try:
        if not os.path.exists(pdf_path):
            logger.error(f"PDF file not found: {pdf_path}")
            # Use a mock document for testing if file not found
            return ["Ceci est un document de test pour le droit tunisien."]
            
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
        documents = text_splitter.split_documents(pages)
        logger.info(f"Successfully loaded {len(documents)} document chunks")
        return [doc.page_content for doc in documents]
    except Exception as e:
        logger.error(f"Error loading documents: {e}")
        logger.error(traceback.format_exc())
        # Return dummy data to allow testing even if PDF loading fails
        return ["Ceci est un document de test pour le droit tunisien."]

# Create and save vector store
def create_and_save_vector_store(texts, vector_store_path):
    logger.info(f"Creating vector store at {vector_store_path}")
    try:
        embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
        vector_store = Chroma.from_texts(texts, embedding=embeddings, persist_directory=vector_store_path)
        vector_store.persist()
        logger.info("Vector store created and saved successfully")
        return vector_store
    except Exception as e:
        logger.error(f"Error creating vector store: {e}")
        logger.error(traceback.format_exc())
        raise

# Load vector store
def load_vector_store(vector_store_path):
    try:
        if os.path.exists(vector_store_path):
            logger.info(f"Loading existing vector store from {vector_store_path}")
            embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
            return Chroma(persist_directory=vector_store_path, embedding_function=embeddings)
        logger.info("No existing vector store found")
        return None
    except Exception as e:
        logger.error(f"Error loading vector store: {e}")
        logger.error(traceback.format_exc())
        return None

# Create BM25 index
def create_bm25_index(texts):
    logger.info("Creating BM25 index")
    try:
        tokenized_corpus = [text.split() for text in texts]
        return BM25Okapi(tokenized_corpus), texts
    except Exception as e:
        logger.error(f"Error creating BM25 index: {e}")
        logger.error(traceback.format_exc())
        raise

# Simple test function for the LLM
def test_llm(llm_instance):
    logger.info("Testing LLM with a simple query")
    try:
        test_result = llm_instance("Test query: What is your name?")
        logger.info(f"LLM test result: {test_result}")
        return True
    except Exception as e:
        logger.error(f"LLM test failed: {e}")
        logger.error(traceback.format_exc())
        return False

# Primary processing function - replaced the FastAPI endpoint logic
def process_query(query, llm, prompt_template, vector_store, bm25, bm25_texts):
    logger.info(f"Processing query: {query}")
    
    try:
        # BM25 retrieval - with safety checks
        if bm25 is None or bm25_texts is None:
            logger.error("BM25 index not initialized")
            return {"error": "BM25 index not initialized"}
            
        try:
            bm25_scores = bm25.get_scores(query.split())
            top_k = min(3, len(bm25_scores))
            if top_k <= 0:
                top_bm25 = []
            else:
                top_bm25_indices = torch.topk(torch.tensor(bm25_scores), top_k).indices.tolist()
                top_bm25 = [bm25_texts[i] for i in top_bm25_indices]
            logger.info(f"Retrieved {len(top_bm25)} documents from BM25")
        except Exception as e:
            logger.error(f"Error in BM25 retrieval: {e}")
            top_bm25 = []
        
        # Chroma dense retrieval - with safety checks
        if vector_store is None:
            logger.error("Vector store not initialized")
            return {"error": "Vector store not initialized"}
            
        try:
            retriever = vector_store.as_retriever(search_kwargs={"k": 3})
            chroma_results = retriever.get_relevant_documents(query)
            top_dense = [doc.page_content for doc in chroma_results]
            logger.info(f"Retrieved {len(top_dense)} documents from dense retrieval")
        except Exception as e:
            logger.error(f"Error in dense retrieval: {e}")
            top_dense = []
        
        # Combine contexts - with empty list check
        if not top_bm25 and not top_dense:
            context = "Aucun contexte juridique pertinent trouvé."
        else:
            # Combine and truncate context
            combined_context = "\n".join(top_bm25 + top_dense)
            context = truncate_text_to_token_limit(combined_context, max_tokens=256)
        
        logger.info(f"Final context length: {len(context)} characters")
        
        # Use the template to format the prompt
        try:
            formatted_prompt = prompt_template.format(
                context=context,
                question=query
            )
            logger.info(f"Formatted prompt length: {len(formatted_prompt)}")
        except Exception as e:
            logger.error(f"Error formatting prompt: {e}")
            return {"error": f"Error formatting prompt: {str(e)}"}
        
        # Generate answer
        logger.info("Generating answer with LLM")
        try:
            # Handle timeout for LLM
            response = llm(formatted_prompt)
            logger.info(f"Raw LLM response: {response}")
            
            # Handle different response formats
            if isinstance(response, list) and len(response) > 0:
                if isinstance(response[0], dict) and 'generated_text' in response[0]:
                    answer = response[0]['generated_text']
                else:
                    answer = str(response[0])
            else:
                answer = str(response)
                
            # Log and prepare response
            logger.info("Query processed successfully")
            return {
                "answer": answer.strip(),
                "citations": (top_bm25 + top_dense)[:3]  # Limit to 3 citations
            }
        except Exception as e:
            logger.error(f"Error generating answer: {e}")
            logger.error(traceback.format_exc())
            return {
                "error": f"Error generating answer: {str(e)}",
                "context_used": context[:200] + "..."  # Include part of the context for debugging
            }
    except Exception as e:
        logger.error(f"Unhandled error processing query: {e}")
        logger.error(traceback.format_exc())
        return {"error": str(e)}

# Main function
def main():
    try:
        # Initialize LLM and prompt template
        llm = load_llm()
        prompt_template = create_legal_prompt_template()
        
        # Test LLM
        # if not test_llm(llm):
        #     logger.error("LLM test failed, using fallback mode")
            
        # Load and process data
        pdf_path = r"Tunisia_Crim-Fr.pdf"  # Update this path to your PDF location
        texts = load_documents(pdf_path)
        
        vector_store = load_vector_store(vector_store_path)
        if vector_store is None:
            vector_store = create_and_save_vector_store(texts, vector_store_path)
        
        bm25, bm25_texts = create_bm25_index(texts)
        logger.info("All components initialized successfully")
        
        # Sample queries to test
        queries = [
            "Dans quelles conditions le tribunal peut-il décider de placer le condamné sous surveillance administrative selon l'article 25 ?",
            "Quelle est la durée maximale de surveillance administrative imposée par l'article 25 et à quel type d'infractions s'applique-t-elle ?",
            "Pour quelles infractions la surveillance administrative est-elle encourue de plein droit pendant dix ans selon l'article 26 ?",
            "Existe-t-il des exceptions à la durée de dix ans de surveillance administrative prévue par l'article 26, ou cette durée s'applique-t-elle systématiquement ?"
        ]

        logger.info("Testing sample queries")
        for q in queries:
            print(f"\nQuestion : {q}")
            result = process_query(q, llm, prompt_template, vector_store, bm25, bm25_texts)
            
            if "error" in result:
                print(f"Error: {result['error']}")
            else:
                print("Réponse :", result["answer"])
                if "citations" in result and result["citations"]:
                    print("Citation :", result["citations"][0][:100] + "...")  # Show start of first citation
                else:
                    print("Aucune citation disponible")
        
        # Interactive mode
        print("\n\nMode interactif (tapez 'q' pour quitter):")
        while True:
            user_query = input("\nEntrez votre question juridique: ")
            if user_query.lower() == 'q':
                break
                
            result = process_query(user_query, llm, prompt_template, vector_store, bm25, bm25_texts)
            if "error" in result:
                print(f"Error: {result['error']}")
            else:
                print("\nRéponse :", result["answer"])
                if "citations" in result and result["citations"]:
                    print("\nCitations:")
                    for i, citation in enumerate(result["citations"], 1):
                        print(f"{i}. {citation[:150]}...")
                else:
                    print("\nAucune citation disponible")
                    
    except Exception as e:
        logger.critical(f"Critical error during execution: {e}")
        logger.critical(traceback.format_exc())
        return False

if __name__ == "__main__":
    main()


2025-04-05 16:57:45,966 - __main__ - INFO - Loading language model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
2025-04-05 16:57:45,970 - __main__ - INFO - Using device: cpu


Device set to use cpu
2025-04-05 16:57:54,932 - __main__ - INFO - LLM loaded successfully
2025-04-05 16:57:54,958 - __main__ - INFO - Creating legal QA prompt template.
2025-04-05 16:57:54,963 - __main__ - INFO - Loading documents from Tunisia_Crim-Fr.pdf
2025-04-05 16:57:56,102 - __main__ - INFO - Successfully loaded 427 document chunks
2025-04-05 16:57:56,104 - __main__ - INFO - Loading existing vector store from vector_store
2025-04-05 16:57:56,129 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-04-05 16:57:56,129 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2025-04-05 17:06:28,633 - __main__ - INFO - Creating BM25 index
2025-04-05 17:06:28,667 - __main__ - INFO - All components initialized successfully
2025-04-05 17:06:28,668 - __main__ - INFO - Testing sample queries
2025-04-05 17:06:28,669 - __main__ - INFO - Processing query: Dans quelles conditions le trib


Question : Dans quelles conditions le tribunal peut-il décider de placer le condamné sous surveillance administrative selon l'article 25 ?


2025-04-05 17:06:30,024 - __main__ - INFO - Retrieved 3 documents from dense retrieval
2025-04-05 17:06:30,636 - __main__ - INFO - Final context length: 778 characters
2025-04-05 17:06:30,641 - __main__ - INFO - Formatted prompt length: 1098
2025-04-05 17:06:30,642 - __main__ - INFO - Generating answer with LLM
2025-04-05 17:09:20,338 - __main__ - INFO - Raw LLM response: 
    <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> Imprimerie Officielle de la République Tunisienne
16 
Article 23 
Le renvoi sous la surveillance administrative reconnaît à 
l’autorité administrative le droit de déterminer le lieu de 
résidence du condamné à l'expiration de sa peine et celui de le 
modifier, si elle le juge utile. 
Article 24 
Le condamné ne peut, sans autorisation, quitter la résidence 
qui lui a été assignée. 
Article 25 (Modifié par le décret du 22 octobre 1940). 
Lorsque l'infraction comporte une peine supér

Réponse : <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> Imprimerie Officielle de la République Tunisienne
16 
Article 23 
Le renvoi sous la surveillance administrative reconnaît à 
l’autorité administrative le droit de déterminer le lieu de 
résidence du condamné à l'expiration de sa peine et celui de le 
modifier, si elle le juge utile. 
Article 24 
Le condamné ne peut, sans autorisation, quitter la résidence 
qui lui a été assignée. 
Article 25 (Modifié par le décret du 22 octobre 1940). 
Lorsque l'infraction comporte une peine supérieure à deux
ans de prison ou constitue une deuxième récidive, le tribunal 
peut ordonner que le condamné soit placé sous la surveillance 
administrative pour une période dont le maximum ne dépasse 
pas cinq ans. 
Article 26 (Modifié par la loi n° 66-63 du 5 juillet 1966). 
A moins que le
    Question: Dans quelles conditions le tribunal peut-il décider de placer le c

...

Réponse : <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> Imprimerie Officielle de la République Tunisienne
16 
Article 23 
Le renvoi sous la surveillance administrative reconnaît à 
l’autorité administrative le droit de déterminer le lieu de 
résidence du condamné à l'expiration de sa peine et celui de le 
modifier, si elle le juge utile. 
Article 24 
Le condamné ne peut, sans autorisation, quitter la résidence 
qui lui a été assignée. 
Article 25 (Modifié par le décret du 22 octobre 1940). 
Lorsque l'infraction comporte une peine supérieure à deux
Imprimerie Officielle de la République Tunisienne
58 
Section IX - Des enfreintes  à l'interdiction de séjour ou à la 
surveillance administrative 
Article 150 
Est puni de l'emprisonnement pendant un an, le condamné 
qui contrevient à l'interdiction de séjour ou qui, pla
    Question: Quelle est la durée maximale de surveillance administrative imposée pa

2025-04-05 17:10:15,797 - __main__ - INFO - Retrieved 3 documents from dense retrieval
2025-04-05 17:10:16,628 - __main__ - INFO - Final context length: 765 characters
2025-04-05 17:10:16,630 - __main__ - INFO - Formatted prompt length: 1083
2025-04-05 17:10:16,632 - __main__ - INFO - Generating answer with LLM
2025-04-05 17:11:29,993 - __main__ - INFO - Raw LLM response: 
    <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> ans de prison ou constitue une deuxième récidive, le tribunal 
peut ordonner que le condamné soit placé sous la surveillance 
administrative pour une période dont le maximum ne dépasse 
pas cinq ans. 
Article 26 (Modifié par la loi n° 66-63 du 5 juillet 1966). 
A moins que le tribunal n’en ait autrement ordonnée, la 
surveillance administratif est encourue de plein droit pendant 
dix années en cas de condamnation prononcée en application 
des articles 60 à 79 ou 231 à 235 du prése

Réponse : <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> ans de prison ou constitue une deuxième récidive, le tribunal 
peut ordonner que le condamné soit placé sous la surveillance 
administrative pour une période dont le maximum ne dépasse 
pas cinq ans. 
Article 26 (Modifié par la loi n° 66-63 du 5 juillet 1966). 
A moins que le tribunal n’en ait autrement ordonnée, la 
surveillance administratif est encourue de plein droit pendant 
dix années en cas de condamnation prononcée en application 
des articles 60 à 79 ou 231 à 235 du présent code ou pour
Imprimerie Officielle de la République Tunisienne
58 
Section IX - Des enfreintes  à l'interdiction de séjour ou à la 
surveillance administrative 
Article 150 
Est puni de l'emprisonnement pendant un an, le condamné 
qui contrevient à l'interdiction de séjour
    Question: Pour quelles infractions la surveillance administrative est-elle encourue de pl

2025-04-05 17:11:30,240 - __main__ - INFO - Retrieved 3 documents from dense retrieval
2025-04-05 17:11:30,655 - __main__ - INFO - Final context length: 765 characters
2025-04-05 17:11:30,656 - __main__ - INFO - Formatted prompt length: 1114
2025-04-05 17:11:30,658 - __main__ - INFO - Generating answer with LLM
2025-04-05 17:12:15,216 - __main__ - INFO - Raw LLM response: 
    <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> Imprimerie Officielle de la République Tunisienne
21 
Toutefois, lorsque la peine encourue est la peine de mort ou 
l'emprisonnement à vie, elle est remplacée par un 
emprisonnement de dix ans. 
Si la peine encourue est celle de l'emprisonnement pour une 
durée déterminée, cette durée est réduite de moitié, sans que la 
peine prononcée ne dépasse cinq ans. 
Les peines complémentaires énoncées à l’article 5 du présent 
code ne sont pas applicables, il en est de même des règles de 


Réponse : <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> Imprimerie Officielle de la République Tunisienne
21 
Toutefois, lorsque la peine encourue est la peine de mort ou 
l'emprisonnement à vie, elle est remplacée par un 
emprisonnement de dix ans. 
Si la peine encourue est celle de l'emprisonnement pour une 
durée déterminée, cette durée est réduite de moitié, sans que la 
peine prononcée ne dépasse cinq ans. 
Les peines complémentaires énoncées à l’article 5 du présent 
code ne sont pas applicables, il en est de même des règles de 
récidive.
une contravention.  La peine d'un jour d'emprisonnement est de 
vingt quatre heures, celle d'un mois est de trente jours. 
Article 15 
La durée de toute peine privative de liberté compte du jour 
où le condamné est détenu en vertu d’une condamnation 
devenue définit
    Question: Existe-t-il des exceptions à la durée de dix ans de surveillance administrative

In [ ]:
import os
import torch
import logging
import sys
import traceback

import gradio as gr

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Model configuration
LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Publicly available model
vector_store_path = "vector_store"
# Embedding model configuration (using a 768-dimensional model)
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"

# Model loading functions
def load_llm():
    """Load the language model."""
    logger.info(f"Loading language model: {LLM_MODEL}")
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Using device: {device}")
        tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            logger.info("Set pad_token to eos_token as it was None")
        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None,
            trust_remote_code=True
        )
        # Create text generation pipeline with default parameters.
        gen_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.5,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            device=0 if device == "cuda" else -1
        )
        logger.info("LLM loaded successfully")
        return HuggingFacePipeline(pipeline=gen_pipeline)
    except Exception as e:
        logger.error(f"Error loading LLM: {e}")
        logger.error(traceback.format_exc())
        raise

def create_legal_prompt_template():
    """Create a refined prompt template for legal QA."""
    logger.info("Creating legal QA prompt template.")
    template = """
    <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: {context}
    Question: {question}
    </|user|>
    
    <|assistant|>
    """
    return PromptTemplate(input_variables=["context", "question"], template=template)

def truncate_text_to_token_limit(text, max_tokens=256):
    try:
        tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
        encoded = tokenizer.encode(text)
        if len(encoded) > max_tokens:
            encoded = encoded[:max_tokens]
        return tokenizer.decode(encoded)
    except Exception as e:
        logger.error(f"Error truncating text: {e}")
        return text[:max_tokens * 4]

def load_documents(pdf_path):
    logger.info(f"Loading documents from {pdf_path}")
    try:
        if not os.path.exists(pdf_path):
            logger.error(f"PDF file not found: {pdf_path}")
            return ["Ceci est un document de test pour le droit tunisien."]
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
        documents = text_splitter.split_documents(pages)
        logger.info(f"Successfully loaded {len(documents)} document chunks")
        return [doc.page_content for doc in documents]
    except Exception as e:
        logger.error(f"Error loading documents: {e}")
        logger.error(traceback.format_exc())
        return ["Ceci est un document de test pour le droit tunisien."]

def create_and_save_vector_store(texts, vector_store_path):
    logger.info(f"Creating vector store at {vector_store_path}")
    try:
        embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
        vector_store = Chroma.from_texts(texts, embedding=embeddings, persist_directory=vector_store_path)
        vector_store.persist()
        logger.info("Vector store created and saved successfully")
        return vector_store
    except Exception as e:
        logger.error(f"Error creating vector store: {e}")
        logger.error(traceback.format_exc())
        raise

def load_vector_store(vector_store_path):
    try:
        if os.path.exists(vector_store_path):
            logger.info(f"Loading existing vector store from {vector_store_path}")
            embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
            return Chroma(persist_directory=vector_store_path, embedding_function=embeddings)
        logger.info("No existing vector store found")
        return None
    except Exception as e:
        logger.error(f"Error loading vector store: {e}")
        logger.error(traceback.format_exc())
        return None

def create_bm25_index(texts):
    logger.info("Creating BM25 index")
    try:
        tokenized_corpus = [text.split() for text in texts]
        return BM25Okapi(tokenized_corpus), texts
    except Exception as e:
        logger.error(f"Error creating BM25 index: {e}")
        logger.error(traceback.format_exc())
        raise

def process_query_with_params(query, llm, prompt_template, vector_store, bm25, bm25_texts, gen_params):
    logger.info(f"Processing query: {query}")
    try:
        # BM25 retrieval
        if bm25 is None or bm25_texts is None:
            logger.error("BM25 index not initialized")
            return {"error": "BM25 index not initialized"}
        try:
            bm25_scores = bm25.get_scores(query.split())
            top_k = min(3, len(bm25_scores))
            if top_k <= 0:
                top_bm25 = []
            else:
                top_bm25_indices = torch.topk(torch.tensor(bm25_scores), top_k).indices.tolist()
                top_bm25 = [bm25_texts[i] for i in top_bm25_indices]
            logger.info(f"Retrieved {len(top_bm25)} documents from BM25")
        except Exception as e:
            logger.error(f"Error in BM25 retrieval: {e}")
            top_bm25 = []
        # Chroma dense retrieval
        if vector_store is None:
            logger.error("Vector store not initialized")
            return {"error": "Vector store not initialized"}
        try:
            retriever = vector_store.as_retriever(search_kwargs={"k": 3})
            chroma_results = retriever.get_relevant_documents(query)
            top_dense = [doc.page_content for doc in chroma_results]
            logger.info(f"Retrieved {len(top_dense)} documents from dense retrieval")
        except Exception as e:
            logger.error(f"Error in dense retrieval: {e}")
            top_dense = []
        if not top_bm25 and not top_dense:
            context = "Aucun contexte juridique pertinent trouvé."
        else:
            combined_context = "\n".join(top_bm25 + top_dense)
            context = truncate_text_to_token_limit(combined_context, max_tokens=256)
        logger.info(f"Final context length: {len(context)} characters")
        try:
            formatted_prompt = prompt_template.format(context=context, question=query)
            logger.info(f"Formatted prompt length: {len(formatted_prompt)}")
        except Exception as e:
            logger.error(f"Error formatting prompt: {e}")
            return {"error": f"Error formatting prompt: {str(e)}"}
        logger.info("Generating answer with LLM")
        try:
            # Pass generation parameters (temperature, max_new_tokens, etc.) to the LLM call.
            response = llm(formatted_prompt, **gen_params)
            logger.info(f"Raw LLM response: {response}")
            if isinstance(response, list) and len(response) > 0:
                if isinstance(response[0], dict) and 'generated_text' in response[0]:
                    answer = response[0]['generated_text']
                else:
                    answer = str(response[0])
            else:
                answer = str(response)
            logger.info("Query processed successfully")
            return {
                "answer": answer.strip(),
                "citations": (top_bm25 + top_dense)[:3]
            }
        except Exception as e:
            logger.error(f"Error generating answer: {e}")
            logger.error(traceback.format_exc())
            return {"error": f"Error generating answer: {str(e)}", "context_used": context[:200] + "..."}
    except Exception as e:
        logger.error(f"Unhandled error processing query: {e}")
        logger.error(traceback.format_exc())
        return {"error": str(e)}

# UI Chatbot function using Gradio
def chatbot_ui(user_query, temperature, max_new_tokens):
    # Generation parameters from the UI
    gen_params = {"temperature": temperature, "max_new_tokens": int(max_new_tokens)}
    logger.info(f"User query: {user_query} | Gen params: {gen_params}")
    result = process_query_with_params(
        user_query, global_llm, global_prompt_template,
        global_vector_store, global_bm25, global_bm25_texts,
        gen_params
    )
    if "error" in result:
        return f"Error: {result['error']}"
    response = result["answer"]
    # Optionally, include the first citation snippet if available.
    if result.get("citations"):
        response += "\n\nCitations:\n" + "\n".join([f"{i+1}. {cit[:150]}..." for i, cit in enumerate(result["citations"])])
    return response

def initialize_components():
    # Initialize LLM, prompt, documents, vector store, and BM25 index.
    global global_llm, global_prompt_template, global_vector_store, global_bm25, global_bm25_texts
    global_llm = load_llm()
    global_prompt_template = create_legal_prompt_template()
    pdf_path = r"Tunisia_Crim-Fr.pdf"  # Update this path as needed
    texts = load_documents(pdf_path)
    global_vector_store = load_vector_store(vector_store_path)
    if global_vector_store is None:
        global_vector_store = create_and_save_vector_store(texts, vector_store_path)
    global_bm25, global_bm25_texts = create_bm25_index(texts)
    logger.info("All components initialized successfully.")

def launch_chatbot_ui():
    initialize_components()
    with gr.Blocks() as demo:
        gr.Markdown("## Chatbot Juridique - Droit Tunisien")
        with gr.Row():
            user_query = gr.Textbox(label="Votre question juridique", lines=2, placeholder="Entrez votre question ici...")
        with gr.Row():
            temperature = gr.Slider(0.0, 1.0, value=0.5, step=0.05, label="Temperature")
            max_new_tokens = gr.Slider(50, 512, value=256, step=1, label="Max New Tokens")
        output = gr.Textbox(label="Réponse", lines=10)
        submit_btn = gr.Button("Envoyer")
        submit_btn.click(chatbot_ui, inputs=[user_query, temperature, max_new_tokens], outputs=output)
    demo.launch()

if __name__ == "__main__":
    launch_chatbot_ui()


C:\Users\mouni\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-05 17:22:19,443 - __main__ - INFO - Loading language model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
2025-04-05 17:22:19,444 - __main__ - INFO - Using device: cpu


Device set to use cpu


2025-04-05 17:22:29,233 - __main__ - INFO - LLM loaded successfully


C:\Users\mouni\AppData\Local\Temp\ipykernel_24560\2068329624.py:62: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=gen_pipeline)


...

2025-04-05 17:22:46,235 - httpx - INFO - HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
2025-04-05 17:22:46,851 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-04-05 17:22:46,873 - httpx - INFO - HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
2025-04-05 17:24:04,432 - __main__ - INFO - User query: Quelle est la durée maximale de surveillance administrative imposée par l'article 25 et à quel type d'infractions s'applique-t-elle ? répondre en francais svp | Gen params: {'temperature': 0.5, 'max_new_tokens': 256}
2025-04-05 17:24:04,442 - __main__ - INFO - Processing query: Quelle est la durée maximale de surveillance administrative imposée par l'article 25 et à quel type d'infractions s'applique-t-elle ? répondre en francais svp
2025-04-05 17:24:04,514 - __main__ - INFO - Retrieved 3 documents from BM25


C:\Users\mouni\AppData\Local\Temp\ipykernel_24560\2068329624.py:174: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_results = retriever.get_relevant_documents(query)


2025-04-05 17:24:06,133 - __main__ - INFO - Retrieved 3 documents from dense retrieval
2025-04-05 17:24:06,700 - __main__ - INFO - Final context length: 774 characters
2025-04-05 17:24:06,701 - __main__ - INFO - Formatted prompt length: 1125
2025-04-05 17:24:06,704 - __main__ - INFO - Generating answer with LLM


C:\Users\mouni\AppData\Local\Temp\ipykernel_24560\2068329624.py:195: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_prompt, **gen_params)


2025-04-05 17:25:14,449 - __main__ - INFO - Raw LLM response: 
    <|system|>
    Vous êtes un assistant juridique spécialisé dans le droit tunisien.
    </|system|>
    
    <|user|>
    Contexte: <s> Imprimerie Officielle de la République Tunisienne
16 
Article 23 
Le renvoi sous la surveillance administrative reconnaît à 
l’autorité administrative le droit de déterminer le lieu de 
résidence du condamné à l'expiration de sa peine et celui de le 
modifier, si elle le juge utile. 
Article 24 
Le condamné ne peut, sans autorisation, quitter la résidence 
qui lui a été assignée. 
Article 25 (Modifié par le décret du 22 octobre 1940). 
Lorsque l'infraction comporte une peine supérieure à deux
Imprimerie Officielle de la République Tunisienne
58 
Section IX - Des enfreintes  à l'interdiction de séjour ou à la 
surveillance administrative 
Article 150 
Est puni de l'emprisonnement pendant un an, le condamné 
qui contrevient à l'interdiction de séjour ou qui, pla
    Question: Quelle est la